### Read in relevant data sets

In [1]:
import os
from openpyxl import load_workbook
import pandas as pd
from datetime import timedelta

BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('')))
DATA_DIR = os.path.join(BASE_DIR, 'amerigo_island', 'data')

xls_filename = 'batteries.xlsm'
input_ws_name = 'Basic Inputs'
header_col_num = 6 

wb = load_workbook(os.path.join(DATA_DIR, xls_filename), data_only=True)
input_ws = wb[input_ws_name]

input_ws_vals = [row for row in input_ws.values]
input_ws_headers = input_ws_vals[header_col_num]

orig_load_and_gen_df = pd.DataFrame(data= input_ws_vals[header_col_num + 1:], columns= input_ws_headers)

### Create variables for all columns of dataframes

In [2]:
date_colname = 'date'
load_colname = 'load_mw'
solar_gen_colname = 'solar_mw'
wind_gen_colname = 'wind_mw'
total_vre_gen_colname = 'total_vre_mw'
day_of_month_colname = 'day_of_month'
day_of_yr_colname = 'day_of_yr'
week_of_yr_colname = 'wk_of_yr'
month_colname = 'month'
year_colname = 'year'
weekday_colname = 'weekday'
daily_load_vre_diff_colname = 'daily_load_vre_diff'
prev_7_load_colname = 'prev_7_load'
prev_7_load_vre_diff_colname = 'prev_7_load_vre_diff'
prev_7_vre_gen_colname = 'prev_7_vre_gen'
critical_load_mw_colname = 'critical_load_mw'
critical_load_less_vre_colname = 'critical_load_less_vre_mw'
cuml_load_since_prev_charge_colname = 'cuml_load_since_prev_charge_mw'
cuml_charge_since_prev_charge_colname = 'cuml_charge_since_prev_discharge_mw'
charge_surplus_colname = 'charge_surplus'
battery_soc_colname = 'battery_soc'

critical_load_perc = .33

input_ws_cols = (date_colname, load_colname, wind_gen_colname, solar_gen_colname)

### Standardize column names 
* add columns derived from prev cols

In [3]:
load_and_gen_df = orig_load_and_gen_df.copy()
load_and_gen_df = load_and_gen_df.iloc[:, [i for i in range(len(input_ws_cols))]]
load_and_gen_df.columns = input_ws_cols

### Clean vals in existing cols

In [4]:
# drop all rows that have all NaNs
load_and_gen_df = load_and_gen_df.dropna(how='all')

# all solar vals need to be > 0
load_and_gen_df[solar_gen_colname] = load_and_gen_df[solar_gen_colname]\
    .apply(lambda x: x if x > 0 else 0)

# bringing in date col from xlsx causes date to be x:59:59 instead of x+1:00:00
load_and_gen_df[date_colname] = load_and_gen_df[date_colname].map(lambda x: x + timedelta(seconds=1))

# add wind and solar to get total VRE / hr
load_and_gen_df[total_vre_gen_colname] = load_and_gen_df[wind_gen_colname] + load_and_gen_df[solar_gen_colname]
load_and_gen_df[critical_load_mw_colname] = load_and_gen_df[load_colname] * critical_load_perc
load_and_gen_df[critical_load_less_vre_colname] = load_and_gen_df[critical_load_mw_colname] - load_and_gen_df[total_vre_gen_colname]
load_and_gen_df[charge_surplus_colname] = - load_and_gen_df[critical_load_less_vre_colname]

load_and_gen_df[critical_load_less_vre_colname] = load_and_gen_df[critical_load_less_vre_colname]\
    .map(lambda x: x if x > 0 else 0)

load_and_gen_df[charge_surplus_colname] = load_and_gen_df[charge_surplus_colname]\
    .map(lambda x: x if x > 0 else 0)

In [5]:
load_and_gen_df

,date,load_mw,wind_mw,solar_mw,total_vre_mw,critical_load_mw,critical_load_less_vre_mw,charge_surplus
0,2017-01-01 00:00:01.000000,35.1,1.67966,0.0,1.67966,11.583,9.90334,0.0
1,2017-01-01 01:00:01.000000,35.1,1.67966,0.0,1.67966,11.583,9.90334,0.0
2,2017-01-01 02:00:01.000000,34.6,1.23327,0.0,1.23327,11.418,10.18473,0.0
3,2017-01-01 03:00:00.999999,34.2,1.09193,0.0,1.09193,11.286,10.19407,0.0
4,2017-01-01 04:00:00.999999,33.8,1.46510,0.0,1.46510,11.154,9.68890,0.0
...,...,...,...,...,...,...,...,...
8755,2017-12-31 19:00:00.998165,45.5,6.67250,0.0,6.67250,15.015,8.34250,0.0
8756,2017-12-31 20:00:00.998165,46.5,6.76534,0.0,6.76534,15.345,8.57966,0.0
8757,2017-12-31 21:00:00.998165,45.0,6.20556,0.0,6.20556,14.850,8.64444,0.0
8758,2017-12-31 22:00:00.998165,41.7,8.72619,0.0,8.72619,13.761,5.03481,0.0


In [6]:
load_and_gen_df[month_colname] = load_and_gen_df[date_colname].map(lambda x: x.month)
load_and_gen_df[day_of_month_colname] = load_and_gen_df[date_colname].map(lambda x: x.day)
load_and_gen_df[day_of_yr_colname] = load_and_gen_df[date_colname].map(lambda x: x.timetuple().tm_yday)

In [7]:
assert len(load_and_gen_df) == 8760, "Expected one row per hour in year (8760), got {} rows".format(len(load_and_gen_df))

for day_of_yr in load_and_gen_df[day_of_yr_colname].unique():
    if len(load_and_gen_df[load_and_gen_df[day_of_yr_colname] == day_of_yr]) != 24:
        print("Day {} has {} values".format(day_of_yr, len(load_and_gen_df[load_and_gen_df[day_of_yr_colname] == day_of_yr])))

## Determine resilient weeks
<hr/>

##### Three cases:
* Maximum continuous 7-day load
* Maximum continuous 7-day difference bt (`load` - `VRE`)
* Week w maximum instantaneous demand

In [8]:
# group by day
daily_load_df = load_and_gen_df.groupby(day_of_yr_colname)[[load_colname, total_vre_gen_colname]].sum().reset_index()

In [9]:
daily_load_df.head()

,day_of_yr,load_mw,total_vre_mw
0,1,949.3,167.018267
1,2,1014.8,282.558527
2,3,1019.5,190.506975
3,4,1007.8,233.377199
4,5,1001.6,190.717257


In [10]:
daily_load_vre_diff_colname

'daily_load_vre_diff'

In [11]:
# diff bt load and vre gen
daily_load_df[daily_load_vre_diff_colname] = daily_load_df[load_colname] - daily_load_df[total_vre_gen_colname]

# critical load (percentage of total load)
daily_load_df[critical_load_mw_colname] = daily_load_df[load_colname] * critical_load_perc

# critical load minus vre gen
daily_load_df[critical_load_less_vre_colname] = daily_load_df[critical_load_mw_colname] - daily_load_df[total_vre_gen_colname]

# if vre gen > critical load, can use that to charge battery
daily_load_df[charge_surplus_colname] = - daily_load_df[critical_load_less_vre_colname]

# remove negatives of columns that can't go less than 0
daily_load_df[critical_load_less_vre_colname] = daily_load_df[critical_load_less_vre_colname]\
    .map(lambda x: x if x > 0 else 0)
daily_load_df[charge_surplus_colname] = daily_load_df[charge_surplus_colname]\
    .map(lambda x: x if x > 0 else 0)

### Function to create "lookback" columns
For any given day, we want to know the previous week's [load, load - vre, etc.]. `sum_prev_rows` calculates these types of columns. Default is a week lookback, but this can be an argument passed into the function.

In [12]:
def sum_prev_rows(row, df, lookback_colname, agg_colname, days = 7):

    start_idx = row[lookback_colname] - days 
        
    return_val = df.loc[start_idx:row[lookback_colname] - 1, agg_colname].sum() if start_idx >= 0 else None
    
    return return_val

In [13]:
daily_load_df[prev_7_load_colname] = daily_load_df\
    .apply(sum_prev_rows, axis=1, args=(daily_load_df, day_of_yr_colname, load_colname,))

daily_load_df[prev_7_load_vre_diff_colname] = daily_load_df\
    .apply(sum_prev_rows, axis=1, args=(daily_load_df, day_of_yr_colname, daily_load_vre_diff_colname,))

daily_load_df[prev_7_vre_gen_colname] = daily_load_df\
    .apply(sum_prev_rows, axis=1, args=(daily_load_df, day_of_yr_colname, total_vre_gen_colname,))

# critical daily load
daily_load_df[critical_load_mw_colname] = daily_load_df[load_colname] * critical_load_perc 

In [14]:
daily_load_df.head(10)

,day_of_yr,load_mw,total_vre_mw,daily_load_vre_diff,critical_load_mw,critical_load_less_vre_mw,charge_surplus,prev_7_load,prev_7_load_vre_diff,prev_7_vre_gen
0,1,949.3,167.018267,782.281733,313.269,146.250733,0.000000,NaN,NaN,NaN
1,2,1014.8,282.558527,732.241473,334.884,52.325473,0.000000,NaN,NaN,NaN
2,3,1019.5,190.506975,828.993025,336.435,145.928025,0.000000,NaN,NaN,NaN
3,4,1007.8,233.377199,774.422801,332.574,99.196801,0.000000,NaN,NaN,NaN
4,5,1001.6,190.717257,810.882743,330.528,139.810743,0.000000,NaN,NaN,NaN
5,6,1019.5,178.216357,841.283643,336.435,158.218643,0.000000,NaN,NaN,NaN
6,7,977.8,322.880095,654.919905,322.674,0.000000,0.206095,6990.3,5425.025323,1565.274677
7,8,931.5,263.091220,668.408780,307.395,44.303780,0.000000,6972.5,5311.152370,1661.347630
8,9,1009.2,235.635424,773.564576,333.036,97.400576,0.000000,6966.9,5352.475473,1614.424527
9,10,1018.3,310.944086,707.355914,336.039,25.094914,0.000000,6965.7,5230.838362,1734.861638


### Find the day numbers that represent our three resilience cases

In [15]:
resilience_case_load_daynum = daily_load_df[daily_load_df[prev_7_load_colname] == daily_load_df[prev_7_load_colname].max()].to_dict('r')[0][day_of_yr_colname]
resilience_case_vre_load_diff_daynum = daily_load_df[daily_load_df[prev_7_load_vre_diff_colname] == daily_load_df[prev_7_load_vre_diff_colname].max()].to_dict('r')[0][day_of_yr_colname]
resilience_case_max_demand_daynum = load_and_gen_df[load_and_gen_df[load_colname] == load_and_gen_df[load_colname].max()].to_dict('r')[0][day_of_yr_colname]

#resilience_case_max_demand = daily_load_df[daily_load_df[load_colname] == daily_load_df[load_colname].max()]

### Create function that calculates new columns needed to make battery sizing decisions
* What would the battery size be if we sized it to be able to handle the max cumulative discharge in a given week?

In [16]:
def add_storage_input_cols(df, daynum, solar_scaling_factor = 1, wind_scaling_factor = 1):
    
    decorated_df = df.copy()
    
    decorated_df = decorated_df[
        (decorated_df[day_of_yr_colname] <= daynum)
        & (decorated_df[day_of_yr_colname] >= daynum - 6)
    ]
    
    decorated_df = decorated_df.reset_index(drop=True)
    
    decorated_df.sort_values(date_colname)
    decorated_df[cuml_load_since_prev_charge_colname] = None
    
    for idx, row in decorated_df.iterrows():
        
        if idx == 0:
            val = row[critical_load_less_vre_colname]
        elif row[critical_load_less_vre_colname] > 0:
            val = decorated_df.loc[idx - 1, cuml_load_since_prev_charge_colname] + row[critical_load_less_vre_colname]
        else:
            val = 0
            
        decorated_df.at[idx, cuml_load_since_prev_charge_colname] = val
        
    return decorated_df

In [17]:
resil_max_week_load_df = add_storage_input_cols(load_and_gen_df, resilience_case_load_daynum)
resil_vre_load_diff_df = add_storage_input_cols(load_and_gen_df, resilience_case_vre_load_diff_daynum)
resil_max_demand_df = add_storage_input_cols(load_and_gen_df, resilience_case_max_demand_daynum)

In [18]:
max_battery_discharge_keyname = "max_battery_discharge"
max_battery_charge_keyname = "max_battery_charge"
max_capacity_mwh_keyname = "max_capacity_mwh"
batt_power_mw_keyname = "batt_power_mw" 
batt_energy_mwh_keyname = "batt_energy_mwh" 
weekly_battery_discharge_mwh_keyname = "weekly_battery_discharge_mwh" 
excess_gen_avail_mwh_keyname = "excess_gen_avail_mwh"
min_charge_at_blackout_keyname = "min_charge_at_blackout"

def get_battery_characteristics(df):

    max_battery_discharge = df[critical_load_less_vre_colname].max()
    max_battery_charge = df[charge_surplus_colname].max()
    batt_power_mw = max([max_battery_charge, max_battery_discharge])
    
    max_capacity_mwh = df[cuml_load_since_prev_charge_colname].max()
    weekly_battery_discharge_mwh = df[critical_load_less_vre_colname].sum()
    excess_gen_avail_mwh = df[charge_surplus_colname].sum()
    min_charge_at_blackout = weekly_battery_discharge_mwh - excess_gen_avail_mwh
    batt_energy_mwh = max([max_capacity_mwh, min_charge_at_blackout])
    
    battery_characteristics = {
        max_battery_discharge_keyname : max_battery_discharge,
        max_battery_charge_keyname : max_battery_charge,
        max_capacity_mwh_keyname : max_capacity_mwh,
        batt_power_mw_keyname : batt_power_mw,
        batt_energy_mwh_keyname :batt_energy_mwh,
        weekly_battery_discharge_mwh_keyname : weekly_battery_discharge_mwh,
        excess_gen_avail_mwh_keyname : excess_gen_avail_mwh,
        min_charge_at_blackout_keyname : min_charge_at_blackout
    }
    
    return battery_characteristics


In [19]:
resil_max_week_load_characteristics = get_battery_characteristics(resil_max_week_load_df)
resil_vre_load_diff_characteristics = get_battery_characteristics(resil_vre_load_diff_df)
resil_max_demand_characteristics = get_battery_characteristics(resil_max_demand_df)

In [20]:
def calc_soc_cols(df, config):
    
    decorated_df = df.copy()
    batt_energy_mwh = config[batt_energy_mwh_keyname]
    
    decorated_df[battery_soc_colname] = None
    decorated_df.reset_index(drop=True)
    
    for idx, row in decorated_df.iterrows():
        
        if idx == 0:
            val = batt_energy_mwh - row[critical_load_less_vre_colname] + row[charge_surplus_colname]
        else:
            charge = decorated_df.loc[idx - 1, battery_soc_colname] - row[critical_load_less_vre_colname] + row[charge_surplus_colname] 
            val = charge if charge < batt_energy_mwh else batt_energy_mwh
            
        decorated_df.at[idx, battery_soc_colname] = val
    
    return decorated_df

In [21]:
resil_max_week_load_w_soc_df = calc_soc_cols(resil_max_week_load_df, resil_max_week_load_characteristics)
resil_vre_load_diff_w_soc_df = calc_soc_cols(resil_vre_load_diff_df, resil_vre_load_diff_characteristics)
resil_max_demand_w_soc_df = calc_soc_cols(resil_max_demand_df, resil_max_demand_characteristics)


In [22]:
resil_vre_load_diff_w_soc_df

,date,load_mw,wind_mw,solar_mw,total_vre_mw,critical_load_mw,critical_load_less_vre_mw,charge_surplus,month,day_of_month,day_of_yr,cuml_load_since_prev_charge_mw,battery_soc
0,2017-06-10 00:00:00.999195,40.9,2.536390,0.000000,2.536390,13.497,10.960610,0.0,6,10,161,10.9606,1103.09
1,2017-06-10 01:00:00.999195,38.8,3.192490,0.000000,3.192490,12.804,9.611510,0.0,6,10,161,20.5721,1093.48
2,2017-06-10 02:00:00.999195,41.7,2.712380,0.000000,2.712380,13.761,11.048620,0.0,6,10,161,31.6207,1082.43
3,2017-06-10 03:00:00.999195,40.0,2.267670,0.000000,2.267670,13.200,10.932330,0.0,6,10,161,42.5531,1071.5
4,2017-06-10 04:00:00.999194,36.5,1.263280,0.000000,1.263280,12.045,10.781720,0.0,6,10,161,53.3348,1060.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,2017-06-16 19:00:00.999161,47.9,1.179860,0.112323,1.292183,15.807,14.514817,0.0,6,16,167,31.5701,58.3746
164,2017-06-16 20:00:00.999161,48.2,1.150880,0.000000,1.150880,15.906,14.755120,0.0,6,16,167,46.3253,43.6194
165,2017-06-16 21:00:00.999161,47.3,0.664125,0.000000,0.664125,15.609,14.944875,0.0,6,16,167,61.2701,28.6746
166,2017-06-16 22:00:00.999161,49.1,1.054970,0.000000,1.054970,16.203,15.148030,0.0,6,16,167,76.4182,13.5265
